<a href="https://colab.research.google.com/github/bhushanrajs/sciml_project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Hello World')

Hello World


In [1]:
print("Hello World")

Hello World
